title部分使用 bigru；content 部分使用cnn.

两部分分别使用不同的 embedding.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import pickle
import os
import sys
import shutil
import time


time0 = time.time()
print('Starting ...')
model_name = 'p5-1-2embed-rnn256-cnn2345'                    # 模型名称
W_embedding = np.load('../data/W_embedding.npy').astype(np.float32)            # 导入预训练好的词向量
model_path = '../ckpt/' + model_name + '/'                  # 模型保存位置
summary_path = '../summary/' + model_name + '/'             # summary 位置
result_path = '../result/' + model_name + '.csv'            # result.csv 位置
scores_path = '../scores/' + model_name + '.npy'            # scores.npy 位置

if not os.path.exists(model_path):
    os.makedirs(model_path)         
model_path = model_path + 'model.ckpt'
if os.path.exists(summary_path):   # 删除原来的 summary 文件，避免重合
    shutil.rmtree(summary_path)
os.makedirs(summary_path)          # 然后再次创建


# ##################### config ######################
n_step1 = max_len1 = 50           # 句子长度
n_step2 = max_len2 = 150
input_size = embedding_size = 256       # 字向量长度

# title-bigru
hidden_size = 256    # 隐含层节点数
n_layer = 1        # bi-gru 层数
# content-cnn
filter_sizes = [2,3,4,5]                  # 卷积核大小
n_filter = 256                          # 每种卷积核的个数
n_filter_total = n_filter * len(filter_sizes)
# output
fc_hidden_size = 1024                   # fc 层节点数
n_class = 1999

global_step = 0
tr_batch_size = 128
te_batch_size = 128
print('Prepared, costed time %g s.' % (time.time() - time0))

Starting ...
Prepared, costed time 0.347928 s.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from tensorflow.contrib import rnn
import tensorflow.contrib.layers as layers

'''
双端 GRU，知乎问题多标签分类。
'''

print('Building model ...')
lr = tf.placeholder(tf.float32)
keep_prob = tf.placeholder(tf.float32, [])
batch_size = tf.placeholder(tf.int32, [])  # 注意类型必须为 tf.int32
tst = tf.placeholder(tf.bool)
n_updates = tf.placeholder(tf.int32)      # training iteration,传入 bn 层
update_emas = list()   # BN 层中所有的更新操作


def weight_variable(shape, name, initializer=None):
    """Create a weight variable with appropriate initialization."""
    if initializer is None:
        initializer = tf.contrib.layers.xavier_initializer()
    return tf.get_variable(name=name, shape=shape, initializer=initializer, dtype=tf.float32)

def bias_variable(shape, name):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.get_variable(name=name, initializer=initial, dtype=tf.float32)


def batchnorm(Ylogits, is_test, num_updates, offset, convolutional=False):
    """batchnormalization.
    Args:
        Ylogits: 1D向量或者是3D的卷积结果。
        num_updates: 迭代的global_step
        offset：表示beta，全局均值；在 RELU 激活中一般初始化为 0.1。
        scale：表示lambda，全局方差；在 sigmoid 激活中需要，这 RELU 激活中作用不大。
        m: 表示batch均值；v:表示batch方差。
        bnepsilon：一个很小的浮点数，防止除以 0.
    Returns:
        Ybn: 和 Ylogits 的维度一样，就是经过 Batch Normalization 处理的结果。
        update_moving_everages：更新mean和variance，主要是给最后的 test 使用。
    """
    exp_moving_avg = tf.train.ExponentialMovingAverage(0.999, num_updates) # adding the iteration prevents from averaging across non-existing iterations
    bnepsilon = 1e-5
    if convolutional:
        mean, variance = tf.nn.moments(Ylogits, [0, 1, 2])
    else:
        mean, variance = tf.nn.moments(Ylogits, [0])
    update_moving_everages = exp_moving_avg.apply([mean, variance])
    m = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean)
    v = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance)
    Ybn = tf.nn.batch_normalization(Ylogits, m, v, offset, None, bnepsilon)
    return Ybn, update_moving_everages

# 第一次定义，放在 CPU 上面
with tf.device('/gpu:0'):
    with tf.variable_scope('title_embed'):
        title_embedding = tf.get_variable(name="title_embedding", shape=W_embedding.shape, 
                        initializer=tf.constant_initializer(W_embedding), trainable=True)   # fine-tune
    with tf.variable_scope('content_embed'):
        content_embedding = tf.get_variable(name="content_embedding", shape=W_embedding.shape, 
                        initializer=tf.constant_initializer(W_embedding), trainable=True)   # fine-tune
        
with tf.name_scope('Inputs'):
    X1_inputs = tf.placeholder(tf.int64, [None, n_step1], name='X1_input')
    X2_inputs = tf.placeholder(tf.int64, [None, n_step2], name='X2_input')
    y_inputs = tf.placeholder(tf.float32, [None, n_class], name='y_input')
    inputs1 = tf.nn.embedding_lookup(title_embedding, X1_inputs)
    inputs2 = tf.nn.embedding_lookup(content_embedding, X2_inputs)

def gru_cell():
    with tf.name_scope('gru_cell'):
        cell = rnn.GRUCell(hidden_size, reuse=tf.get_variable_scope().reuse)
    return rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)


def bi_gru(inputs):
    """build the bi-GRU network. 返回个最后一层的隐含状态。"""      
    cells_fw = [gru_cell() for _ in range(n_layer)]
    cells_bw = [gru_cell() for _ in range(n_layer)]
    initial_states_fw = [cell_fw.zero_state(batch_size, tf.float32) for cell_fw in cells_fw]
    initial_states_bw = [cell_bw.zero_state(batch_size, tf.float32) for cell_bw in cells_bw] 
    outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(cells_fw, cells_bw, inputs, 
                        initial_states_fw = initial_states_fw, initial_states_bw = initial_states_bw, dtype=tf.float32)
    return outputs
    

def task_specific_attention(inputs, output_size,
                            initializer=layers.xavier_initializer(),
                            activation_fn=tf.tanh, scope=None):
    """
    Performs task-specific attention reduction, using learned
    attention context vector (constant within task of interest).
    Args:
        inputs: Tensor of shape [batch_size, units, input_size]
            `input_size` must be static (known)
            `units` axis will be attended over (reduced from output)
            `batch_size` will be preserved
        output_size: Size of output's inner (feature) dimension
    Returns:
        outputs: Tensor of shape [batch_size, output_dim].
    """
    assert len(inputs.get_shape()) == 3 and inputs.get_shape()[-1].value is not None
    with tf.variable_scope(scope or 'attention') as scope:
        # u_w, attention 向量
        attention_context_vector = tf.get_variable(name='attention_context_vector',
                                                   shape=[output_size],
                                                   initializer=initializer,
                                                   dtype=tf.float32)
        # 全连接层，把 h_i 转为 u_i ， shape= [batch_size, units, input_size] -> [batch_size, units, output_size]
        input_projection = layers.fully_connected(inputs, output_size,
                                                  activation_fn=activation_fn,
                                                  scope=scope)
        # 输出 [batch_size, units]
        vector_attn = tf.reduce_sum(tf.multiply(input_projection, attention_context_vector), axis=2, keep_dims=True)
        attention_weights = tf.nn.softmax(vector_attn, dim=1)   
        tf.summary.histogram('attention_weigths', attention_weights)
        weighted_projection = tf.multiply(inputs, attention_weights)   # ???源代码,原文应该是对 hi 加权！！
        outputs = tf.reduce_sum(weighted_projection, axis=1)
        return outputs   # 输出 [batch_size, hidden_size*2]


def textcnn(inputs, n_step):
    """build the TextCNN network. Return the h_drop"""
    # inputs.shape = [batchsize, n_step, embedding_size]
    inputs = tf.expand_dims(inputs, -1)
    pooled_outputs = list()
    for i, filter_size in enumerate(filter_sizes):
        with tf.variable_scope("conv-maxpool-%s" % filter_size):
            # Convolution Layer
            filter_shape = [filter_size, embedding_size, 1, n_filter]
            W_filter = weight_variable(shape=filter_shape, name='W_filter')
            beta = bias_variable(shape=[n_filter], name='beta_filter')
            tf.summary.histogram('beta_filter', beta)
            conv = tf.nn.conv2d(inputs, W_filter, strides=[1, 1, 1, 1], padding="VALID", name="conv")
            conv_bn, update_ema = batchnorm(conv, tst, n_updates, beta, convolutional=True)    # 在激活层前面加 BN
            # Apply nonlinearity, batch norm scaling is not useful with relus
            # batch norm offsets are used instead of biases,使用 BN 层的 offset，不要 biases
            h = tf.nn.relu(conv_bn, name="filter_relu")
            # Maxpooling over the outputs
            pooled = tf.nn.max_pool(h,ksize=[1, n_step - filter_size + 1, 1, 1],
                strides=[1, 1, 1, 1],padding='VALID',name="pool")
            pooled_outputs.append(pooled)
            update_emas.append(update_ema)
    h_pool = tf.concat(pooled_outputs, 3)
    h_pool_flat = tf.reshape(h_pool, [-1, n_filter_total]) 
    return h_pool_flat    # shape = [-1, n_filter_total]


# title-bigru 部分
with tf.variable_scope('bigru_title'):          # 两部分的输出都加上 BN+RELU 层，
    word_encoder_title = bi_gru(inputs1)      # title 部分输出
    output_title = task_specific_attention(word_encoder_title, hidden_size*2)
    
# content-cnn 部分    
with tf.variable_scope('cnn-content'):
    output_content = textcnn(inputs2, n_step2)
    

with tf.name_scope('fc_layer'): # 将两个网络的输出拼接后加一个 fc 层
    output = tf.concat([output_title, output_content], axis=1)
    W_fc = weight_variable([hidden_size*2+n_filter_total, fc_hidden_size], name='Weight_fc')
    tf.summary.histogram('W_fc', W_fc)
    h_fc = tf.matmul(output, W_fc, name='h_fc')
    beta_fc = tf.Variable(tf.constant(0.1, tf.float32, shape=[fc_hidden_size], name="beta_fc"))
    tf.summary.histogram('beta_fc', beta_fc)
    fc_bn, update_ema_fc = batchnorm(h_fc, tst, n_updates, beta_fc, convolutional=False)
    update_emas.append(update_ema_fc)
    fc_bn_relu = tf.nn.relu(fc_bn, name="relu")
    
    
with tf.name_scope('out_layer'):
    W_out = weight_variable([fc_hidden_size, n_class], name='Weight_out') 
    tf.summary.histogram('Weight_out', W_out)
    b_out = bias_variable([n_class], name='bias_out') 
    tf.summary.histogram('bias_out', b_out)
    y_pred = tf.nn.xw_plus_b(fc_bn_relu, W_out, b_out, name='y_pred')  #每个类别的分数 scores

with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred, labels=y_inputs))
    tf.summary.scalar('cost', cost)
    
# ***** 优化求解 *******
with tf.name_scope('AdamOptimizer1'):
    tvars1 = tf.trainable_variables()
    grads1 = tf.gradients(cost, tvars1)
    optimizer1 = tf.train.AdamOptimizer(learning_rate=lr)
    train_op1 = optimizer1.apply_gradients(zip(grads1, tvars1),
        global_step=tf.contrib.framework.get_or_create_global_step())
    
with tf.name_scope('AdamOptimizer2'):
    tvars2 = [tvar for tvar in tvars1 if 'title_embed' not in tvar.name]
    grads2 = tf.gradients(cost, tvars2)
    optimizer2 = tf.train.AdamOptimizer(learning_rate=lr)
    train_op2 = optimizer2.apply_gradients(zip(grads2, tvars2),
        global_step=tf.contrib.framework.get_or_create_global_step())
update_op = tf.group(*update_emas)   # 更新 BN 参数
   
# summary
merged = tf.summary.merge_all() # summary
train_writer = tf.summary.FileWriter(summary_path + 'train', sess.graph)
test_writer = tf.summary.FileWriter(summary_path + 'test')
print 'Finished creating the bi-gru model.'

Building model ...
Finished creating the bi-gru model.


## 导入数据

In [3]:
sys.path.append('..')
from data_helpers import BatchGenerator
from data_helpers import to_categorical
from evaluator import score_eval


data_train_path = '/home/huangyongye/zhihu_data/data_train/'
data_valid_path = '/home/huangyongye/zhihu_data/data_valid/'

# data_train_path = '../data256/data_train/'
# data_valid_path = '../data256/data_valid/'

tr_batches = os.listdir(data_train_path)   # batch 文件名列表
va_batches = os.listdir(data_valid_path)
n_tr_batches = len(tr_batches)
n_va_batches = len(va_batches)


# 测试用
# n_tr_batches = 500  
# n_va_batches = 20
print('n_tr_batch=%d' % n_tr_batches)
print('n_va_batches=%d' % n_va_batches)


def get_batch(data_path, batch_id, title_len=n_step1):
    """get a batch from data_path"""
    new_batch = np.load(data_path + str(batch_id) + '.npz')
    X_batch = new_batch['X']
    y_batch = new_batch['y']
    X1_batch = X_batch[:, :title_len]
    X2_batch = X_batch[:, title_len:]
    return [X1_batch, X2_batch, y_batch]


def valid_epoch(data_path=data_valid_path):
    """Test on the valid data."""
    _costs = 0.0
    predict_labels_list = list()  # 所有的预测结果
    marked_labels_list = list()   # 真实标签
    for i in xrange(n_va_batches):
        [X1_batch, X2_batch, y_batch] = get_batch(data_path, i)
        marked_labels_list.extend(y_batch)
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        fetches = [merged, cost, y_pred]  
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch,  y_inputs:y_batch, lr:_lr,
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        summary, _cost, predict_labels = sess.run(fetches, feed_dict)
        _costs += _cost
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    mean_cost = _costs / n_va_batches
    return mean_cost, precision, recall, f1

print('Every thing prepared!')

n_tr_batch=22656
n_va_batches=782
Every thing prepared!


## 训练模型

In [4]:
# 测试
# max_epoch = 1
# max_max_epoch = 2 
# decay_step = 500
# valid_step = 250

# 正式 
max_epoch = 2
max_max_epoch = 7 
decay_step = 10000
valid_step = 8000


_lr = 6e-4
decay1 = 0.95
decay2 = 0.70
print('n_tr_batches=%d' % n_tr_batches)
saver = tf.train.Saver(max_to_keep=3)           # 最多保存的模型数量
sess.run(tf.global_variables_initializer())
last_f1 = 0.40
model_num = 0
global_step = 0

n_tr_batches=22656


In [5]:
time0 = time.time()
train_op = train_op2 
decay = decay1

for epoch in xrange(max_max_epoch):
    batch_indexs = np.random.permutation(n_tr_batches)  # shuffle the training data
    if epoch == max_epoch:
        train_op = train_op1
        decay = decay2   # 增加学习率衰减
        if model_num == 0:
            model_num += 1
            save_path = saver.save(sess, model_path, global_step=model_num)
            print('the save path is ', save_path)
        print('Begin updating embedding.')
    print 'EPOCH %d, lr= %g' % (epoch+1, _lr)    
    for batch in xrange(n_tr_batches): 
        global_step += 1
        if (global_step+1) % valid_step == 0:    # 进行 valid 计算
            valid_cost, precision, recall, f1 = valid_epoch()
            print('Global_step=%d: valid cost=%g; p=%g, r=%g, f1=%g, time=%g s' % (
                    global_step, valid_cost, precision, recall, f1, time.time()-time0))
            time0 = time.time()
            if (f1 > last_f1):
                last_f1 = f1
                model_num += 1
                save_path = saver.save(sess, model_path, global_step=model_num)
                print('the save path is ', save_path) 
        
                
        batch_id = batch_indexs[batch]
        [X1_batch, X2_batch, y_batch] = get_batch(data_train_path, batch_id, n_step1)
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        fetches = [merged, cost, train_op, update_op]
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, y_inputs:y_batch, batch_size:_batch_size, lr:_lr, 
                     keep_prob:0.5, tst:False, n_updates:global_step}
        summary, _cost, _, _ = sess.run(fetches, feed_dict) # the cost is the mean cost of one batch
        if global_step % 100:
            train_writer.add_summary(summary, global_step)
            batch_id = np.random.randint(0, n_va_batches)   # 随机选一个验证batch
            [X1_batch, X2_batch, y_batch] = get_batch(data_valid_path, batch_id, n_step1)
            y_batch = to_categorical(y_batch)
            _batch_size = len(y_batch)
            feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch,  y_inputs:y_batch, lr:_lr,
                         batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
            fetches = [merged, cost]
            summary, _cost = sess.run(fetches, feed_dict)
            test_writer.add_summary(summary, global_step)

valid_cost, precision, recall, f1 = valid_epoch()  # # 每个 epoch 进行一次验证 valid
print('Global_step=%d;  valid cost=%g; p=%g, r=%g, f1=%g; speed=%g s/epoch' % (
    global_step, valid_cost, precision, recall, f1, time.time()-time0) )
if (f1 > last_f1):
    model_num += 1
    save_path = saver.save(sess, model_path, global_step=model_num)
    print('the save path is ', save_path) 

EPOCH 1, lr= 0.0006
Global_step=7999: valid cost=0.00656199; p=1.05316, r=0.439179, f1=0.309934, time=3377.96 s
Global_step=15999: valid cost=0.00641636; p=1.17828, r=0.489345, f1=0.345752, time=3369.46 s
EPOCH 2, lr= 0.0006
Global_step=23999: valid cost=0.00497349; p=1.23817, r=0.511886, f1=0.362161, time=3255.98 s
Global_step=31999: valid cost=0.0051245; p=1.27635, r=0.525175, f1=0.372077, time=3254.38 s
Global_step=39999: valid cost=0.00499167; p=1.29814, r=0.533679, f1=0.378198, time=3297.67 s
('the save path is ', '../ckpt/p5-1-2embed-rnn256-cnn2345/model.ckpt-1')
Begin updating embedding.
EPOCH 3, lr= 0.0006


KeyboardInterrupt: 

In [6]:
# 上面忘记进行学习率衰减了

time0 = time.time()
train_op = train_op1 
decay = decay2
_lr = 0.00042

for epoch in xrange(max_max_epoch):
    batch_indexs = np.random.permutation(n_tr_batches)  # shuffle the training data
    print 'EPOCH %d, lr= %g' % (epoch+1, _lr)    
    for batch in xrange(n_tr_batches): 
        global_step += 1
        if (global_step+1) % valid_step == 0:    # 进行 valid 计算
            valid_cost, precision, recall, f1 = valid_epoch()
            print('Global_step=%d: valid cost=%g; p=%g, r=%g, f1=%g, time=%g s' % (
                    global_step, valid_cost, precision, recall, f1, time.time()-time0))
            time0 = time.time()
            if (f1 > last_f1):
                last_f1 = f1
                model_num += 1
                save_path = saver.save(sess, model_path, global_step=model_num)
                print('the save path is ', save_path) 
        if (global_step+1) % decay_step == 0:    # 进行 valid 计算
            _lr = _lr * decay
                
        batch_id = batch_indexs[batch]
        [X1_batch, X2_batch, y_batch] = get_batch(data_train_path, batch_id, n_step1)
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        fetches = [merged, cost, train_op, update_op]
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, y_inputs:y_batch, batch_size:_batch_size, lr:_lr, 
                     keep_prob:0.5, tst:False, n_updates:global_step}
        summary, _cost, _, _ = sess.run(fetches, feed_dict) # the cost is the mean cost of one batch
        if global_step % 100:
            train_writer.add_summary(summary, global_step)
            batch_id = np.random.randint(0, n_va_batches)   # 随机选一个验证batch
            [X1_batch, X2_batch, y_batch] = get_batch(data_valid_path, batch_id, n_step1)
            y_batch = to_categorical(y_batch)
            _batch_size = len(y_batch)
            feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch,  y_inputs:y_batch, lr:_lr,
                         batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
            fetches = [merged, cost]
            summary, _cost = sess.run(fetches, feed_dict)
            test_writer.add_summary(summary, global_step)

valid_cost, precision, recall, f1 = valid_epoch()  # # 每个 epoch 进行一次验证 valid
print('Global_step=%d;  valid cost=%g; p=%g, r=%g, f1=%g; speed=%g s/epoch' % (
    global_step, valid_cost, precision, recall, f1, time.time()-time0) )
if (f1 > last_f1):
    model_num += 1
    save_path = saver.save(sess, model_path, global_step=model_num)
    print('the save path is ', save_path) 

EPOCH 1, lr= 0.00042
Global_step=47999: valid cost=0.00412699; p=1.33951, r=0.548597, f1=0.3892, time=587.627 s
Global_step=55999: valid cost=0.00406092; p=1.36636, r=0.559243, f1=0.396825, time=3648.51 s
Global_step=63999: valid cost=0.00397268; p=1.38687, r=0.566815, f1=0.402368, time=3651.12 s
('the save path is ', '../ckpt/p5-1-2embed-rnn256-cnn2345/model.ckpt-2')
EPOCH 2, lr= 0.0002058
Global_step=71999: valid cost=0.00393276; p=1.39665, r=0.570334, f1=0.404964, time=3672.91 s
('the save path is ', '../ckpt/p5-1-2embed-rnn256-cnn2345/model.ckpt-3')
Global_step=79999: valid cost=0.00394946; p=1.39411, r=0.569428, f1=0.404293, time=3698.13 s
Global_step=87999: valid cost=0.00393922; p=1.40149, r=0.572083, f1=0.406252, time=3643.97 s
('the save path is ', '../ckpt/p5-1-2embed-rnn256-cnn2345/model.ckpt-4')
EPOCH 3, lr= 7.05894e-05
Global_step=95999: valid cost=0.00395317; p=1.39837, r=0.570753, f1=0.405319, time=3644.17 s
Global_step=103999: valid cost=0.00395813; p=1.3995, r=0.570937

KeyboardInterrupt: 

每个 epoch 2899952/256=11327 个batch；大模型需要 8409 MB 显存

- Bi-LSTM 模型<br/>
**batch_size=256，迭代12个epoch，基本收敛。结果： 验证集 f1=0.38618； 提交结果 0.3873186**
- Bi-GRU 模型<br/>
**batch_size=256，迭代15个epoch。大概在 13 个 epoch 就收敛了。结果： 验证集 f1=0.390534； 提交结果 0.39198**
- attention-Bi-GRU 模型<br/>
**batch_size=256，迭代18个epoch。在16个epoch收敛。结果：验证集 f1=0.391734 ；提交结果 0.39310**
- attention-bigru-title-content-256 模型<br/>
**batch_size=256，迭代11个epoch收敛。结果，验证集 f1=0.396892；提交结果 0.399367 **
- attention-bigru-title-content-512 模型<br/>
**batch_size=256，迭代9个epoch收敛。结果，验证集 f1=0.401345；提交结果  0.4030 **

## 若没有收敛，继续迭代

In [7]:
valid_cost, precision, recall, f1 = valid_epoch()  # # 每个 epoch 进行一次验证 valid
print('Global_step=%d;  valid cost=%g; p=%g, r=%g, f1=%g; speed=%g s/epoch' % (
    global_step, valid_cost, precision, recall, f1, time.time()-time0) )
if (f1 > last_f1):
    model_num += 1
    save_path = saver.save(sess, model_path, global_step=model_num)
    print('the save path is ', save_path) 

Global_step=106151;  valid cost=0.00396243; p=1.39757, r=0.570231, f1=0.404989; speed=1384.4 s/epoch


## 本地测试

In [8]:
# 导入保存好的模型
saver = tf.train.Saver()
best_model_path = model_path + '-' +str(4)
saver.restore(sess, best_model_path)

from tqdm import tqdm

local_scores_path = '../local_scores/' + model_name + '.npy'

# 导入测试数据
def local_predict(scores_path=local_scores_path, data_path=data_valid_path):
    """预测  valid 结果，并保存预测概率 到  scores.csv 文件中。"""
    print('local predicting ...')
    time0 = time.time()
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    marked_labels_list = list()   # 真实标签
    predict_scores = list()
    for i in tqdm(xrange(n_va_batches)):
        [X1_batch, X2_batch, y_batch] = get_batch(data_path, i)
        marked_labels_list.extend(y_batch)
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, y_inputs:y_batch,
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    print('local valid p=%g, r=%g, f1=%g; speed=%g s/epoch' % ( precision, recall, f1, time.time()-time0) )
    np.save(local_scores_path, predict_scores)
    print('Writed the scores into %s, time %g s' % (local_scores_path, time.time()-time0))
    
local_predict()

INFO:tensorflow:Restoring parameters from ../ckpt/p5-1-2embed-rnn256-cnn2345/model.ckpt-4


  0%|          | 1/782 [00:00<01:30,  8.61it/s]

local predicting ...


100%|██████████| 782/782 [01:22<00:00,  9.43it/s]


local valid p=1.40149, r=0.572083, f1=0.406252; speed=83.479 s/epoch
Writed the scores into ../local_scores/p5-1-2embed-rnn256-cnn2345.npy, time 84.1208 s


## 对测试数据进行预测

In [11]:
save_path = '../data/'

# 导入测试数据
def predict():
    """预测测试集结果，并保存到  result.csv 文件中。"""
    data_path = '/home/huangyongye/zhihu_data/data_test/'
    n_te_batches = len(os.listdir(data_path))
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    predict_scores = list()
    for i in tqdm(xrange(n_te_batches)):
        X_batch = np.load(data_path + str(i) + '.npy')
        X1_batch = X_batch[:, :n_step1]
        X2_batch = X_batch[:, n_step1:]
        _batch_size = len(X_batch)
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch,
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    return predict_labels_list, np.vstack(predict_scores)
    
def write_result(predict_labels_list, result_path):
    """把结果写到 result.csv 中"""
    eval_question = np.load('../data/eval_question.npy')
    with open('../data/sr_topic2id.pkl', 'rb') as inp:
        sr_topic2id = pickle.load(inp)
        sr_id2topic = pickle.load(inp)
    pred_labels = np.asarray(predict_labels_list).reshape([-1])
    pred_topics = sr_id2topic[pred_labels].values.reshape([-1, 5])   # 转为 topic
    df_result = pd.DataFrame({'question':eval_question, 'tid0': pred_topics[:,0], 'tid1':pred_topics[:, 1],
                         'tid2': pred_topics[:,2], 'tid3':pred_topics[:,3],'tid4': pred_topics[:,4]})
    df_result.to_csv(result_path, index=False, header=False)
    print('Finished writing the result')
    return df_result

In [12]:
# 预测
time0 = time.time()
print('Predicting the result, writing into %s' % result_path)
predict_labels_list,predict_scores = predict()
df_result = write_result(predict_labels_list, result_path=result_path) 
print(len(df_result))  # 结果应该为 217360
print('Saving the predict_scores into %s' % scores_path)
print(predict_scores.shape)
np.save(scores_path, predict_scores)
print('Finished! costed time %g s' % (time.time()-time0))

  0%|          | 1/1699 [00:00<04:04,  6.94it/s]

Predicting the result, writing into ../result/p5-1-2embed-rnn256-cnn2345.csv


100%|██████████| 1699/1699 [02:41<00:00,  4.99it/s]


Finished writing the result
217360
Saving the predict_scores into ../scores/p5-1-2embed-rnn256-cnn2345.npy
(217360, 1999)
Finished! costed time 172.151 s


In [10]:
print(np.vstack(predict_scores).shape)
print(predict_scores[0].shape)
print(predict_scores[0])

(217360, 1999)
(1999,)
[ -7.26552773  -7.63365507  -9.62081528 ..., -14.73565578 -19.18001938
  -8.43935585]


In [13]:
df_result.head(15)

,question,tid0,tid1,tid2,tid3,tid4
0,6215603645409872328,4610596224687453206,2919247920214845195,-7506384235581390893,-6839713564940654454,-6306904715218704629
1,6649324930261961840,2858911571784840089,3418451812342379591,-240041917918953337,3383016985780045156,2382911985227044227
2,-4251899610700378615,2919247920214845195,-7358589937244777363,-7270992690764838239,-5385537249890202479,2816249700493135244
3,6213817087034420233,-8655945395761165989,5804619920623030604,-4966205278807386328,2802807297246536023,2787171473654490487
4,-8930652370334418373,-8963554618409314978,3972493657017129406,-1115593437686158905,6018641953300645757,8690965822342756180
5,-2893445476547593888,-6206436693745657677,2131451573312950491,-2696736445927423374,-8274522839089381384,-3900318848301728120
6,2614833994648160978,4482402820945758152,-3856154743789177934,-6748914495015758455,119806503996110046,1553849747662134917
7,1572988006266661060,3738968195649774859,-839691564858654120,7739004195693774975,-2627298052801704596,-8932546057542867495
8,-3736249911643942320,-7653703019053330516,8690965822342756180,1127459907694805235,5502528845814007324,2787171473654490487
9,-976507019126932319,3195914392210930723,3804601920633030746,6940355838132160535,6718676536613592056,2936563632577263124


In [11]:
# 参考结果
df_result.head(5)

,question,tid0,tid1,tid2,tid3,tid4
0,6215603645409872328,4610596224687453206,-6839713564940654454,5818382893362053755,-7506384235581390893,-5932391056759866388
1,6649324930261961840,3418451812342379591,2858911571784840089,-240041917918953337,2382911985227044227,-7483543763655495143
2,-4251899610700378615,2919247920214845195,-7358589937244777363,2816249700493135244,-7270992690764838239,-3315241959305847628
3,6213817087034420233,5804619920623030604,-4966205278807386328,-8655945395761165989,244937959911721367,7476760589625268543
4,-8930652370334418373,3972493657017129406,-8963554618409314978,-1115593437686158905,-6925670792665757873,6018641953300645757


## 在全部预测正确的情况下，理论值为：f1=0.713933
precision=2.50273, recall=0.998873, f1=0.713933

In [26]:
# 假设全部正确，f1 值最高能到多少
def padding_label(labels):
    """把所有的label补齐到长度为 5"""
    label_len = len(labels)
    if label_len >= 5:
        return labels[:5]
    return np.hstack([labels, np.zeros(5-label_len, dtype=int) - 1])
    

marked_labels_list = data_valid.y.tolist() # 所有的标注结果
predict_labels_list = map(padding_label, marked_labels_list)
predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
print '在全部预测正确的情况下，理论值为：'
print 'precision=%g, recall=%g, f1=%g' % (precision, recall, f1)

在全部预测正确的情况下，理论值为：
precision=2.50273, recall=0.998873, f1=0.713933
